In [42]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("white")

In [107]:
data = pd.read_csv('last_10_minute.csv', names = ['Event', 'Time', 'Tx_type', 'Tx_id', 'Accept_id', 'time'])

In [108]:
grouped_data = data.groupby('Tx_id')
tx_id = list(grouped_data.groups.keys())
len(tx_id)

632

In [109]:
#new data frame to get the count of each transaction type
data_unique_ids = data.drop_duplicates(subset = ['Tx_id'])

rfq_count = 0
create_count = 0
bid_count = 0
accept_count = 0
for i in range(0, len(data_unique_ids)):
    if data_unique_ids.iloc[i]['Tx_type'] == 'REQUEST_FOR_QUOTE':
        rfq_count += 1   
    if data_unique_ids.iloc[i]['Tx_type'] == 'CREATE':
        create_count += 1 
    if data_unique_ids.iloc[i]['Tx_type'] == 'BID':
        bid_count += 1   
    if data_unique_ids.iloc[i]['Tx_type'] == 'ACCEPT':
        accept_count += 1   


print(f"#ACCEPTs =  {accept_count}")
print(f"#BIDs =  {bid_count}")
print(f"#RFQs =  {rfq_count}")
print(f"#CREATEs =  {create_count}")
total = rfq_count + create_count + bid_count + accept_count
print(f"#total =  {total}")

#ACCEPTs =  12
#BIDs =  296
#RFQs =  0
#CREATEs =  0
#total =  308


# Throughput

In [71]:
first_10_minute= max(data["time"]) - min(data["time"])
throughput_first = total/first_10_minute
throughput_first

0.36833333333333335

In [102]:
middle_10_minute= max(data["time"]) - min(data["time"])
throughput_middle = total/middle_10_minute
throughput_middle

0.2

In [110]:
last_10_minute= max(data["time"]) - min(data["time"])
throughput_last = total/last_10_minute
throughput_last

0.5124792013311148

In [32]:
df_test = grouped_data.get_group("12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4cafea841bc1464707c7dd")

In [33]:
df_test

,Event,Time,Tx_type,Tx_id,Accept_id,time
93,received_tx,54,REQUEST_FOR_QUOTE,12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4caf...,None,1.665371e+09
96,before_tendermint,57,REQUEST_FOR_QUOTE,12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4caf...,None,1.665371e+09
107,check_tx,69,REQUEST_FOR_QUOTE,12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4caf...,None,1.665371e+09
375,check_tx,435,REQUEST_FOR_QUOTE,12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4caf...,None,1.665371e+09
460,deliver_tx,545,REQUEST_FOR_QUOTE,12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4caf...,None,1.665371e+09
545,end_block,640,REQUEST_FOR_QUOTE,12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4caf...,None,1.665371e+09
630,commit_tx,675,REQUEST_FOR_QUOTE,12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4caf...,None,1.665371e+09
8831,deliver_tx,6739032,REQUEST_FOR_QUOTE,12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4caf...,None,1.665378e+09
8916,end_block,6739213,REQUEST_FOR_QUOTE,12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4caf...,None,1.665378e+09
9001,commit_tx,6739236,REQUEST_FOR_QUOTE,12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4caf...,None,1.665378e+09


In [31]:
"REQUEST_FOR_QUOTE" == grouped_data.get_group("12bec7ce46007ce6a1b41b0e4498b74d5efc61b1bb4cafea841bc1464707c7dd")["Tx_type"].to_list()[0]

True

In [34]:
time_all_committed_rfqs = {}
time_passed_rfqs = []
validated_time = []
accepted_time = []
committed_time = []

rfq_underfit = []
rfq_underfit_dict = {}

rfq_overfit = []
rfq_overfit_dict = {}
for i in range(0, len(tx_id)):
    temp_list = []
    if "REQUEST_FOR_QUOTE" == grouped_data.get_group(tx_id[i])["Tx_type"].to_list()[0]:
        temp_list = grouped_data.get_group(tx_id[i])["Time"].to_list()
        if len(temp_list) == 6 and grouped_data.get_group(tx_id[i])["Event"].to_list()[5] == "commit_tx":
            actual_time = max(temp_list) - min(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
            check_time = df[df['Event'] == 'check_tx']['Time'].to_list()[0]
            accepted = check_time - received_time
            accepted_time.append(accepted)
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
        elif len(temp_list) < 6:
            rfq_underfit.append(temp_list)
            rfq_underfit_dict[tx_id[i]] = temp_list
        elif 15 >= len(temp_list) > 6 :
            df_check = grouped_data.get_group(tx_id[i])
            if len(df_check[df_check['Event'] == 'check_tx']['Time'].to_list()) > 1:
                check_time = max(df_check[df_check['Event'] == 'check_tx']['Time'].to_list())
            else:
                check_time = df_check[df_check['Event'] == 'check_tx']['Time'].to_list()[0] 
            if len(df_check[df_check['Event'] == 'deliver_tx']['Time'].to_list()) > 1:
                deliver_tx = min(df_check[df_check['Event'] == 'deliver_tx']['Time'].to_list())
            else:
                deliver_tx = df_check[df_check['Event'] == 'deliver_tx']['Time'].to_list()[0] 
            if len(df_check[df_check['Event'] == 'deliver_tx']['Time'].to_list()) > 1:
                commit_time = min(df_check[df_check['Event'] == 'commit_tx']['Time'].to_list())
            else:
                commit_time = df_check[df_check['Event'] == 'commit_tx']['Time'].to_list()[0] 
            
            actual_time = commit_time - min(temp_list)
            time_all_committed_rfqs[tx_id[i]] = actual_time
            time_passed_rfqs.append(actual_time)
            
            df = grouped_data.get_group(tx_id[i])
            received_time = df[df['Event'] == 'received_tx']['Time'].to_list()[0]
            before_tendermint_time = df[df['Event'] == 'before_tendermint']['Time'].to_list()[0]
            
            validated = before_tendermint_time - received_time
            validated_time.append(validated)
           
            accepted = check_time - received_time
            accepted_time.append(accepted)
            
            commit_time =  df[df['Event'] == 'commit_tx']['Time'].to_list()[0]
            committed = commit_time - received_time
            committed_time.append(committed)
            rfq_overfit.append(temp_list)
            rfq_overfit_dict[tx_id[i]] = temp_list

In [35]:
len(time_passed_rfqs)

100

[543,
 541,
 433,
 504,
 398,
 418,
 561,
 397,
 523,
 629,
 621,
 465,
 649,
 418,
 554,
 423,
 424,
 487,
 647,
 426,
 518,
 489,
 430,
 481,
 433,
 497,
 407,
 512,
 522,
 589,
 602,
 437,
 560,
 414,
 396,
 443,
 478,
 464,
 602,
 579,
 627,
 640,
 549,
 649,
 471,
 707,
 563,
 460,
 515,
 435,
 453,
 592,
 452,
 696,
 649,
 451,
 461,
 486,
 436,
 462,
 558,
 443,
 583,
 396,
 443,
 509,
 571,
 512,
 419,
 417,
 477,
 428,
 539,
 600,
 485,
 443,
 452,
 605,
 403,
 405,
 645,
 407,
 496,
 639,
 532,
 614,
 411,
 706,
 474,
 433,
 144,
 490,
 496,
 423,
 468,
 704,
 411,
 600,
 531,
 618]

In [36]:
# Analysis of RFQ Transaction
if len(time_passed_rfqs) < 2:
    time_rfqs.append(time_passed_rfqs[0])
rfq_mean = "{:.2f}".format(stat.mean(time_passed_rfqs))
rfq_median = stat.median(time_passed_rfqs)
rfq_max = max(time_passed_rfqs)
rfq_min = min(time_passed_rfqs)
rfq_std_dev = "{:.2f}".format(stat.mean(time_passed_rfqs))

print(rfq_mean)
print(rfq_median)
print(rfq_max)
print(rfq_min)
print(rfq_std_dev)
print(f"total RFQs = {len(time_passed_rfqs)}")

rfq_dict = {
               'min': rfq_min,
               'max': rfq_max,
               'average': rfq_mean,
                'median' : rfq_median,
               'standard_deviation' : rfq_std_dev,
               '# of txs' : len(time_passed_rfqs)}

505.98
488.0
707
144
505.98
total RFQs = 100


In [37]:
results = pd.DataFrame(rfq_dict, index = ['RFQ'])
results.to_csv('rfq_analysis.csv', sep=',', encoding='utf-8')
results

,min,max,average,median,standard_deviation,# of txs
RFQ,144,707,505.98,488.0,505.98,100


In [27]:
results.to_csv('rfq_analysis.csv', sep=',', encoding='utf-8')

In [28]:
results

,min,max,average,median,standard_deviation,# of txs
RFQ,323.0,1233.0,621.69,542.0,621.69,32
